In [34]:
### Imports ###
from nntomo.network import model_training
from nntomo.dataset_slices import DatasetSlices
from nntomo.projection_stack import ProjectionStack
from nntomo.utilities import get_MSE_loss

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Training dataset

For the training dataset, simulated STEM projections fo different MOF structures are taken for the inputs and the SIRT reconstruction with 72 projections is used for the real output values. The selected MOF structures are: MOF-5, PCN-61, IRMOF-76, ZIF-4 and MOF-1131.

In [23]:
training_mofs = ['MOF-5', 'PCN-61', 'IRMOF-76', 'ZIF-4', 'MOF-1131']
cell_repetitions = [3, 2, 2, 5, 3] # Chosen so that the number of voxels around the first axis is roughly the same
symetries = [True, True, True, False, False]

training_stacks = []

for mof, cell_repetition, symetry in zip(training_mofs, cell_repetitions, symetries):
    cif_file = f"data/cif_files/{mof}.cif"
    projections = ProjectionStack.from_cif_file(cif_file, 72, 'full', cell_repetition=(cell_repetition,1,1), del_hydrogen=True, symetry=symetry)
    projections = projections.get_resized_proj(512)
    projections.save()
    training_stacks.append(projections)

tif file retrieved for projection angle -90.0°
tif file retrieved for projection angle -87.5°
tif file retrieved for projection angle -85.0°
tif file retrieved for projection angle -82.5°
tif file retrieved for projection angle -80.0°
tif file retrieved for projection angle -77.5°
tif file retrieved for projection angle -75.0°
tif file retrieved for projection angle -72.5°
tif file retrieved for projection angle -70.0°
tif file retrieved for projection angle -67.5°
tif file retrieved for projection angle -65.0°
tif file retrieved for projection angle -62.5°
tif file retrieved for projection angle -60.0°
tif file retrieved for projection angle -57.5°
tif file retrieved for projection angle -55.0°
tif file retrieved for projection angle -52.5°
tif file retrieved for projection angle -50.0°
tif file retrieved for projection angle -47.5°
tif file retrieved for projection angle -45.0°
tif file retrieved for projection angle -42.5°
tif file retrieved for projection angle -40.0°
tif file retr

c:\Users\Admin-tomo\anaconda3\envs\abtem_env\Lib\site-packages\ase\io\cif.py:401: UserWarning: crystal system 'Cubic' is not interpreted for space group Spacegroup(225, setting=1). This may result in wrong setting!
  warnings.warn(


tif file retrieved for projection angle -90.0°
tif file retrieved for projection angle -87.5°
tif file retrieved for projection angle -85.0°
tif file retrieved for projection angle -82.5°
tif file retrieved for projection angle -80.0°
tif file retrieved for projection angle -77.5°
tif file retrieved for projection angle -75.0°
tif file retrieved for projection angle -72.5°
tif file retrieved for projection angle -70.0°
tif file retrieved for projection angle -67.5°
tif file retrieved for projection angle -65.0°
tif file retrieved for projection angle -62.5°
tif file retrieved for projection angle -60.0°
tif file retrieved for projection angle -57.5°
tif file retrieved for projection angle -55.0°
tif file retrieved for projection angle -52.5°
tif file retrieved for projection angle -50.0°
tif file retrieved for projection angle -47.5°
tif file retrieved for projection angle -45.0°
tif file retrieved for projection angle -42.5°
tif file retrieved for projection angle -40.0°
tif file retr

c:\Users\Admin-tomo\anaconda3\envs\abtem_env\Lib\site-packages\ase\io\cif.py:401: UserWarning: crystal system 'Orthorhombic' is not interpreted for space group Spacegroup(61, setting=1). This may result in wrong setting!
  warnings.warn(


tif file retrieved for projection angle -90.0°
tif file retrieved for projection angle -87.5°
tif file retrieved for projection angle -85.0°
tif file retrieved for projection angle -82.5°
tif file retrieved for projection angle -80.0°
tif file retrieved for projection angle -77.5°
tif file retrieved for projection angle -75.0°
tif file retrieved for projection angle -72.5°
tif file retrieved for projection angle -70.0°
tif file retrieved for projection angle -67.5°
tif file retrieved for projection angle -65.0°
tif file retrieved for projection angle -62.5°
tif file retrieved for projection angle -60.0°
tif file retrieved for projection angle -57.5°
tif file retrieved for projection angle -55.0°
tif file retrieved for projection angle -52.5°
tif file retrieved for projection angle -50.0°
tif file retrieved for projection angle -47.5°
tif file retrieved for projection angle -45.0°
tif file retrieved for projection angle -42.5°
tif file retrieved for projection angle -40.0°
tif file retr

c:\Users\Admin-tomo\anaconda3\envs\abtem_env\Lib\site-packages\ase\io\cif.py:401: UserWarning: crystal system 'monoclinic' is not interpreted for space group Spacegroup(14, setting=1). This may result in wrong setting!
  warnings.warn(


tif file retrieved for projection angle -90.0°
tif file retrieved for projection angle -87.5°
tif file retrieved for projection angle -85.0°
tif file retrieved for projection angle -82.5°
tif file retrieved for projection angle -80.0°
tif file retrieved for projection angle -77.5°
tif file retrieved for projection angle -75.0°
tif file retrieved for projection angle -72.5°
tif file retrieved for projection angle -70.0°
tif file retrieved for projection angle -67.5°
tif file retrieved for projection angle -65.0°
tif file retrieved for projection angle -62.5°
tif file retrieved for projection angle -60.0°
tif file retrieved for projection angle -57.5°
tif file retrieved for projection angle -55.0°
tif file retrieved for projection angle -52.5°
tif file retrieved for projection angle -50.0°
tif file retrieved for projection angle -47.5°
tif file retrieved for projection angle -45.0°
tif file retrieved for projection angle -42.5°
tif file retrieved for projection angle -40.0°
tif file retr

In [24]:
# SIRT reconstructions

training_reconstructions = []
for stack in training_stacks:
    sirt_rec = stack.get_SIRT_reconstruction(150)
    training_reconstructions.append(sirt_rec)
    sirt_rec.save()

Start of SIRT reconstruction.
Saving volume...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\volume_files\sirt150_MOF-5-311-72proj-resized512.mrc.
 ID: sirt150_MOF-5-311-72proj-resized512
Start of SIRT reconstruction.
Saving volume...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\volume_files\sirt150_PCN-61-211-72proj-resized512.mrc.
 ID: sirt150_PCN-61-211-72proj-resized512
Start of SIRT reconstruction.
Saving volume...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\volume_files\sirt150_IRMOF-76-211-72proj-resized512.mrc.
 ID: sirt150_IRMOF-76-211-72proj-resized512
Start of SIRT reconstruction.
Saving volume...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\volume_files\sirt150_ZIF-4-

<pre>
The 72 projections:                The SIRT reconstruction (isosurfarce value: 138/255):

<img src="data/gifs/" width="300"/>            <img src="data/gifs/" width="300"/>
</pre>

In [25]:
# Dataset

training_stacks_sub9 = [stack.get_proj_subset(9) for stack in training_stacks]

segmentations = [100/255, 128/255, 100/255, 90/255, 100/255]
training_reconstructions_segm = [rec.get_segmented_volume(segm) for rec, segm in zip(training_reconstructions, segmentations)]

training_dataset = DatasetSlices(training_stacks_sub9, training_reconstructions_segm)
training_dataset.save()

Saving dataset...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\datasets_files\MOF-5-311-72proj-resized512-sub9_sirt150_MOF-5-311-72proj-resized512-segm0.39_bin.pickle.
 ID: MOF-5-311-72proj-resized512-sub9_sirt150_MOF-5-311-72proj-resized512-segm0.39_bin


### Validation Dataset: UiO-66

In [26]:
### Structure for NN training - MOF: projections ###

validation_stack = ProjectionStack.from_cif_file("data/cif_files/UiO-66.cif", 72, 'full', cell_repetition=(4,1,1), del_hydrogen=True, symetry=True)
validation_stack = validation_stack.get_resized_proj(512)
validation_stack.save()

tif file retrieved for projection angle -90.0°
tif file retrieved for projection angle -87.5°
tif file retrieved for projection angle -85.0°
tif file retrieved for projection angle -82.5°
tif file retrieved for projection angle -80.0°
tif file retrieved for projection angle -77.5°
tif file retrieved for projection angle -75.0°
tif file retrieved for projection angle -72.5°
tif file retrieved for projection angle -70.0°
tif file retrieved for projection angle -67.5°
tif file retrieved for projection angle -65.0°
tif file retrieved for projection angle -62.5°
tif file retrieved for projection angle -60.0°
tif file retrieved for projection angle -57.5°
tif file retrieved for projection angle -55.0°
tif file retrieved for projection angle -52.5°
tif file retrieved for projection angle -50.0°
tif file retrieved for projection angle -47.5°
tif file retrieved for projection angle -45.0°
tif file retrieved for projection angle -42.5°
tif file retrieved for projection angle -40.0°
tif file retr

In [27]:
### SIRT reconstruction ###
validation_reconstruction = validation_stack.get_SIRT_reconstruction(150)
validation_reconstruction.save()

Start of SIRT reconstruction.
Saving volume...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\volume_files\sirt150_UiO-66-411-72proj-resized512.mrc.
 ID: sirt150_UiO-66-411-72proj-resized512


In [28]:
validation_stack_sub9 = validation_stack.get_proj_subset(9)
validation_reconstruction_segm = validation_reconstruction.get_segmented_volume(128/255)

validation_dataset = DatasetSlices(validation_stack_sub9, validation_reconstruction_segm)
validation_dataset.save()

Saving dataset...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\datasets_files\UiO-66-411-72proj-resized512-sub9_sirt150_UiO-66-411-72proj-resized512-segm0.50_bin.pickle.
 ID: UiO-66-411-72proj-resized512-sub9_sirt150_UiO-66-411-72proj-resized512-segm0.50_bin


### NN training

In [41]:
#reverse train and valid
training_dataset = DatasetSlices.retrieve("MOF-5-311-72proj-resized512-sub9_sirt150_MOF-5-311-72proj-resized512-segm0.39_bin")
validation_dataset = DatasetSlices.retrieve("UiO-66-411-72proj-resized512-sub9_sirt150_UiO-66-411-72proj-resized512-segm0.50_bin")

network = model_training(training_dataset, validation_dataset, 16, Nstop=25, learning_rate=0.001)

Start of training.
Epoch 1 (n=0)
-------------------------------
Avg MSELoss(): 0.020788 

Epoch 2 (n=0)
-------------------------------
Avg MSELoss(): 0.003032 

Epoch 3 (n=0)
-------------------------------
Avg MSELoss(): 0.001574 

Epoch 4 (n=0)
-------------------------------
Avg MSELoss(): 0.001347 

Epoch 5 (n=0)
-------------------------------
Avg MSELoss(): 0.001317 

Data saved.
Epoch 6 (n=0)
-------------------------------
Avg MSELoss(): 0.001311 

Epoch 7 (n=0)
-------------------------------
Avg MSELoss(): 0.001310 

Epoch 8 (n=0)
-------------------------------
Avg MSELoss(): 0.001310 

Epoch 9 (n=0)
-------------------------------
Avg MSELoss(): 0.001310 

Epoch 10 (n=0)
-------------------------------
Avg MSELoss(): 0.001310 

Data saved.
Epoch 11 (n=0)
-------------------------------
Avg MSELoss(): 0.001310 

Epoch 12 (n=0)
-------------------------------
Avg MSELoss(): 0.001310 

Epoch 13 (n=0)
-------------------------------
Avg MSELoss(): 0.001310 

Epoch 14 (n=0)
--

KeyboardInterrupt: 

### Reconstructions

In [31]:
# generate for reconstruction:
uio67_stack = ProjectionStack.from_cif_file("data/cif_files/UiO-67.cif", 72, 'full', cell_repetition=(3,3,1), del_hydrogen=True, symetry=True)
uio67_stack = uio67_stack.get_resized_proj(512)
uio67_stack.save()


tif file retrieved for projection angle -90.0°
tif file retrieved for projection angle -87.5°
tif file retrieved for projection angle -85.0°
tif file retrieved for projection angle -82.5°
tif file retrieved for projection angle -80.0°
tif file retrieved for projection angle -77.5°
tif file retrieved for projection angle -75.0°
tif file retrieved for projection angle -72.5°
tif file retrieved for projection angle -70.0°
tif file retrieved for projection angle -67.5°
tif file retrieved for projection angle -65.0°
tif file retrieved for projection angle -62.5°
tif file retrieved for projection angle -60.0°
tif file retrieved for projection angle -57.5°
tif file retrieved for projection angle -55.0°
tif file retrieved for projection angle -52.5°
tif file retrieved for projection angle -50.0°
tif file retrieved for projection angle -47.5°
tif file retrieved for projection angle -45.0°
tif file retrieved for projection angle -42.5°
tif file retrieved for projection angle -40.0°
tif file retr

In [33]:
# Reference SIRT reconstruction
sirt_rec72 = uio67_stack.get_SIRT_reconstruction(150)
sirt_rec72.save()

Start of SIRT reconstruction.
Saving volume...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\volume_files\sirt150_UiO-67-331-72proj-resized512.mrc.
 ID: sirt150_UiO-67-331-72proj-resized512


In [32]:
# NN and SIRT reconstructions with 8 projections
uio67_stack_sub8 = uio67_stack.convert_full_tem().get_proj_subset(8)
nn_rec = uio67_stack_sub8.get_NN_reconstruction(network)
nn_rec.save()
sirt_rec = uio67_stack_sub8.get_SIRT_reconstruction(150)
sirt_rec.save()

Start of NN reconstruction.
Reconstruction part 1/2: [████████████████████████████████████████████████████████████] 8/8 Est wait 00:0.0.0

Reconstruction part 2/2: [████████████████████████████████████████████████████████████] 8/8 Est wait 00:0.0.0

Saving volume...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\volume_files\nn_MOF-5-311-72proj-resized512-sub9_sirt150_MOF-5-311-72proj-resized512-segm0.39_bin_8h_UiO-67-331-72proj-resized512-subtem57-sub8.mrc.
 ID: nn_MOF-5-311-72proj-resized512-sub9_sirt150_MOF-5-311-72proj-resized512-segm0.39_bin_8h_UiO-67-331-72proj-resized512-subtem57-sub8
Start of SIRT reconstruction.
Saving volume...
File saved at c:\Users\Admin-tomo\Documents\tomo-reconstruction-alix\github_repository\nn-tomo-reconstruction\scripts\data\volume_files\sirt150_UiO-67-331-72proj-resized512-subtem57-sub8.mrc.
 ID: sirt150_UiO-67-331-72proj-resized512-subtem57-sub8


### Quantitative results

Same method as [Bladt 2015, Electron tomography based on highly limited data using a neural network reconstruction technique].

In [40]:
sirt_threshold = 1/2
nnfbp_threshold = 40/255

print("MSE between SIRT with 72 proj and SIRT with 8 projs (same threshold): "
      f"{get_MSE_loss(sirt_rec72.get_segmented_volume(sirt_threshold), sirt_rec.get_segmented_volume(sirt_threshold))}")

print("MSE between SIRT with 72 proj and NNFBP with 8 projs: "
      f"{get_MSE_loss(sirt_rec72.get_segmented_volume(sirt_threshold), nn_rec.get_segmented_volume(nnfbp_threshold))}")

MSE between SIRT with 72 proj and SIRT with 8 projs (same threshold): 0.004065476357936859
MSE between SIRT with 72 proj and NNFBP with 8 projs: 0.00029417872428894043
